In [ ]:
from hyperopt import  hp, STATUS_OK, Trials, fmin, tpe

lgb_train=lgb.Dataset(X_train,y_train)
lgb_eval=lgb.Dataset(X_test,y_test,reference=lgb_train)
params_dic={
        'boosting_type':'gbdt',
        'objective':'binary',
        'metric':'auc',
        'is_unbalance':True
        }
space={
       'learning_rete':hp.uniform('learning_rete',0.01,0.1),
       'num_leaves':hp.choice('num_leaves',range(10,190,10)),
       'subsample':hp.uniform('subsample',0.5,1),
       'max_depth':hp.choice('max_depth',range(5,10)),
       'feature_fraction':hp.quniform('feature_fraction',0.1,1,0.2),
       'bagging_fraction':hp.quniform('bagging_fraction',0.1,1,0.2),
       'max_bin':hp.choice('max_bin',range(100,1590,5)),
       'reg_alpha':hp.uniform('reg_alpha',0,2),
       'reg_lambda':hp.uniform('reg_lambda',0,2),
       
       'min_data_in_leaf':hp.choice('min_data_in_leaf',range(5,20,1)),
      }
best_acc=0
def f(params):
    global best_acc
    acc=hyperopt_train_test(params,params_dic)
    if acc>best_acc:
        best_acc=acc
        print('new_best:',best_acc,params)
    return {'loss':-best_acc,'status':STATUS_OK}

trials=Trials()
best=fmin(f,space,algo=tpe.suggest,max_evals=10)

In [ ]:
def hyperopt_train_test(params,hyperparams):
    gbm=lgb.train({**params,**hyperparams},lgb_train,num_boost_round=300,valid_sets=lgb_eval,early_stopping_rounds=30,verbose_eval=0)
    pred=gbm.predict(X_test,num_iterration=gbm.best_iteration)
    score=metrics.roc_auc_score(y_test,pred)
    return score